# missing residueを考慮したidentity, positive, coverageを計算

In [3]:
import numpy as np
import pandas as pd
from Bio import SearchIO, SeqIO, SeqRecord
from Bio.Seq import Seq
from pathlib import Path

In [5]:
target = '5M04_A'
data_dir = Path('data') / target
xml_path = data_dir / (target + '.xml')
template = '4CSU_9_1'
pir_path = data_dir / 'filtering' / (template + '.pir')

In [6]:
qresult_list = list(SearchIO.parse(xml_path, 'blast-xml'))

In [7]:
qresult = qresult_list[0]

In [8]:
qresult

QueryResult(id='5m04_A', 15 hits)

In [17]:
for hit in qresult:
    print(hit)
    print(hit[0].ident_num)
    print('start', hit[0].query_start, 'end', hit[0].query_end)

Query: 5m04_A
       mol:protein length:360 GTPase ObgE/CgtA
  Hit: pdb|5M04|A (360)
       Structure of ObgE from Escherichia coli [Escherichia coli DH5[alpha]]
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0     737.26     360          [0:360]                [0:360]
360
start 0 end 360
Query: 5m04_A
       mol:protein length:360 GTPase ObgE/CgtA
  Hit: pdb|4CSU|9 (390)
       Cryo-EM structures of the 50S ribosome subunit bound with ObgE [Escher...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0     692.58     340         [20:360]                [0:340]
340


In [19]:
sample_hit = qresult[2]
sample_hit

Hit(id='pdb|1LNZ|A', query_id='5m04_A', 1 hsps)

In [20]:
sample_hit[0].aln_annotation

{'similarity': 'FVD+  + V  GDGGNG V+FRREKY+PKGGP GGDGG GGDV  E DE L TL D+R++K F+A RG++G S++  G+   D  IKVP GT V D  T + + D+T+HGQR ++A+GG  G GN+RF +  N  P+   NG PG +R ++LEL +LADVG++G P+ GKST +  VS+AKPK+ADY FTTLVP+LG V  D+ +SFV AD+PGLIEGA +G GLG +FL+H+ER RV++H+ID   ++G DP ++   I  EL +Y+  L  +P+ +V NK    D  EA E  +A  E L  +   + ISA +  G+++L ++V   +   P'}

In [22]:
aln_annotation = sample_hit[0].aln_annotation['similarity']
aln_annotation

'FVD+  + V  GDGGNG V+FRREKY+PKGGP GGDGG GGDV  E DE L TL D+R++K F+A RG++G S++  G+   D  IKVP GT V D  T + + D+T+HGQR ++A+GG  G GN+RF +  N  P+   NG PG +R ++LEL +LADVG++G P+ GKST +  VS+AKPK+ADY FTTLVP+LG V  D+ +SFV AD+PGLIEGA +G GLG +FL+H+ER RV++H+ID   ++G DP ++   I  EL +Y+  L  +P+ +V NK    D  EA E  +A  E L  +   + ISA +  G+++L ++V   +   P'

In [23]:
identity = 0
positive = 0
for r in aln_annotation:
    if r == ' ':
        continue
    elif r == '+':
        positive += 1
    else:
        identity += 1
        positive += 1
print(identity, positive)

163 226


In [35]:
pir_file = data_dir / 'filtering' / '1LNZ_A_1.pir'
for i in SeqIO.parse(pir_file, 'pir'):
    print(i)

ID: TARGET
Name: TARGET
Description: sequence:TARGET:1::360:::::
Number of features: 0
/PIR-type=P1
/molecule_type=protein
Seq('MGSSHHHHHHSSGLVPRGSHMKFVDEASILVVAGDGGNGCVSFRREKYIPKGGP...VQA')
ID: 1LNZ
Name: 1LNZ
Description: structure:1LNZ:2:A:332:A::::
Number of features: 0
/PIR-type=P1
/molecule_type=protein
Seq('----------------------FVDQVKVYVKGGDGGNG-VAFRREKYVPKGGP...---')


In [41]:
pir_template_seq = str(list(SeqIO.parse(pir_file, 'pir'))[1].seq)
pir_template_seq

'----------------------FVDQVKVYVKGGDGGNG-VAFRREKYVPKGGPAGGDGGKGGDVVFEVDEGLRTL-DFRYKKHFKAIRGEHG-SKNQHGRNADD-VIKVPPGTVVTDDDTKQVIADLTEHGQRAVIARGGRGGRGNSRFATPANPAPQLSENGEPGKERYIVLELKVLADVGLVGFPSVGKSTLLSVVSSAKPKIADYHFTTLVPNLG-VETDDGRSFV-ADLPGLIEGAHQGVGLGHQFLRHIERTRVIVHVID-SGLEGRDPYDDYLTINQELSEYNLRLTERPQIIVANK----D-PEAAENLEAFKEKLTDDYPVFPISAVTREGLRELLFEVANQLENTP----'

In [53]:
xml_hit_seq = str(sample_hit[0].hit.seq)
xml_hit_seq

'FVDQVKVYVKGGDGGNGXVAFRREKYVPKGGPAGGDGGKGGDVVFEVDEGLRTLXDFRYKKHFKAIRGEHGXSKNQHGRNADDXVIKVPPGTVVTDDDTKQVIADLTEHGQRAVIARGGRGGRGNSRFATPANPAPQLSENGEPGKERYIVLELKVLADVGLVGFPSVGKSTLLSVVSSAKPKIADYHFTTLVPNLGXVETDDGRSFVXADLPGLIEGAHQGVGLGHQFLRHIERTRVIVHVIDXSGLEGRDPYDDYLTINQELSEYNLRLTERPQIIVANK---XDXPEAAENLEAFKEKLTDDYPVFPISAVTREGLRELLFEVANQLENTP'

In [46]:
aln_annotation

'FVD+  + V  GDGGNG V+FRREKY+PKGGP GGDGG GGDV  E DE L TL D+R++K F+A RG++G S++  G+   D  IKVP GT V D  T + + D+T+HGQR ++A+GG  G GN+RF +  N  P+   NG PG +R ++LEL +LADVG++G P+ GKST +  VS+AKPK+ADY FTTLVP+LG V  D+ +SFV AD+PGLIEGA +G GLG +FL+H+ER RV++H+ID   ++G DP ++   I  EL +Y+  L  +P+ +V NK    D  EA E  +A  E L  +   + ISA +  G+++L ++V   +   P'

In [50]:
start = sample_hit[0].query_start
end = sample_hit[0].query_end
print(start, end)

22 356


In [56]:
pir_template_seq_hit_part =  pir_template_seq[start: end]
pir_template_seq_hit_part

'FVDQVKVYVKGGDGGNG-VAFRREKYVPKGGPAGGDGGKGGDVVFEVDEGLRTL-DFRYKKHFKAIRGEHG-SKNQHGRNADD-VIKVPPGTVVTDDDTKQVIADLTEHGQRAVIARGGRGGRGNSRFATPANPAPQLSENGEPGKERYIVLELKVLADVGLVGFPSVGKSTLLSVVSSAKPKIADYHFTTLVPNLG-VETDDGRSFV-ADLPGLIEGAHQGVGLGHQFLRHIERTRVIVHVID-SGLEGRDPYDDYLTINQELSEYNLRLTERPQIIVANK----D-PEAAENLEAFKEKLTDDYPVFPISAVTREGLRELLFEVANQLENTP'

テンプレートのpirの中の配列と，xml中の配列の一致を確認

一致している場所は問題なし．

一致していない場合はmissing residueの可能性あり

xml配列中にあってpir配列にない場合はmissing residue

逆にxml中になく pir配列にある場合がある．(X残基など)　これは無視されるように処理する．


identity, positiveはxmlのmidlineの残基から数えることが可能(アルファベットは一致，+はpositive, 空白は不一致)

In [61]:
for i, (xr, pr) in enumerate(zip(xml_hit_seq, pir_template_seq_hit_part)):
    if xr != pr:
        print(i)
        print(xr, pr, '"' + aln_annotation[i] + '"')

17
X - " "
54
X - " "
71
X - " "
83
X - " "
197
X - " "
208
X - " "
244
X - " "
285
X - " "
287
X - " "


In [76]:
iteration = 1
qresult = qresult_list[iteration - 1]
hit = qresult[1]
print(hit.id)
pir_name = '_'.join(hit.id.split('|')[1: ]) + '_' + str(iteration) + '.pir'
pir_file = data_dir / 'filtering' / pir_name
print(pir_file)
def calc_correct_template_quality(hit, pir_file: Path):
    assert pir_file.exists()
    aln_annotation = hit[0].aln_annotation['similarity']
    hit_seq = str(hit[0].hit.seq)
    ident_num = hit[0].ident_num
    positive_num = hit[0].pos_num
    aln_len = hit[0].aln_span
    start = hit[0].query_start
    end = hit[0].query_end
    pir_template_seq = str(list(SeqIO.parse(pir_file, 'pir'))[1].seq)[start: end]
    not_ident_num = 0
    not_pos_num = 0
    for i, (xr, pr) in enumerate(zip(hit_seq, pir_template_seq)):
        if xr != pr:
            if aln_annotation[i] == ' ':
                continue
            elif aln_annotation[i] == '+':
                not_pos_num += 1
            else:
                not_pos_num += 1
                not_ident_num += 1
            print(i, xr, pr, '"' + aln_annotation[i] + '"')
    if not_pos_num > 0 or not_ident_num > 0:
        print(not_pos_num, not_ident_num)

pdb|4CSU|9
data/5M04_A/filtering/4CSU_9_1.pir


In [74]:
for iteration in range(1, 4):
    qresult = qresult_list[iteration - 1]
    for hit in qresult:
        pir_name = '_'.join(hit.id.split('|')[1: ]) + '_' + str(iteration) + '.pir'
        pir_file = data_dir / 'filtering' / pir_name
        if not pir_file.exists():
            continue
        print(pir_file)
        calc_correct_template_quality(hit, pir_file)

data/5M04_A/filtering/4CSU_9_1.pir
0 M - "M"
1 K - "K"
2 F - "F"
337 V - "V"
338 Q - "Q"
339 A - "A"
6 6
data/5M04_A/filtering/1LNZ_A_1.pir
data/5M04_A/filtering/1UDX_A_1.pir
data/5M04_A/filtering/2DWQ_A_1.pir
26 N - "+"
27 Y - "Y"
28 P - "P"
29 F - "F"
115 M - "+"
5 3
data/5M04_A/filtering/1WXQ_A_1.pir
24 E - "+"
25 I - "+"
26 A - "A"
27 N - "+"
28 Y - "Y"
29 P - "P"
30 F - "F"
31 T - "T"
32 T - "T"
33 I - "+"
78 G - "G"
79 L - "L"
80 V - "+"
82 G - "G"
83 A - "A"
85 E - "E"
86 G - "G"
88 G - "G"
89 L - "L"
90 G - "G"
92 K - "+"
21 15
data/5M04_A/filtering/4A9A_A_1.pir
62 A - "A"
64 D - "+"
65 G - "G"
125 K - "K"
4 3
data/5M04_A/filtering/1JAL_A_1.pir
22 E - "+"
24 A - "A"
25 N - "+"
26 Y - "Y"
4 2
data/5M04_A/filtering/5EE0_A_1.pir
0 K - "K"
2 D - "+"
5 P - "P"
7 E - "+"
9 R - "R"
93 V - "+"
95 G - "G"
96 A - "A"
99 G - "G"
9 6
data/5M04_A/filtering/1NI3_A_1.pir
data/5M04_A/filtering/5ADY_6_1.pir
data/5M04_A/filtering/2E87_A_2.pir
data/5M04_A/filtering/1XZP_A_2.pir
data/5M04_A/filter

In [98]:
def calc_coverage_from_pir(pir_file):
    query_record, template_record = SeqIO.parse(pir_file, 'pir')
    query_seq, template_seq = target_record.seq, template_record.seq
    template_not_gap_indices = np.where(np.array(template_seq) != '-')[0]
    aln_start, aln_end = template_not_gap_indices[0], template_not_gap_indices[-1]
    query_aln_seq_array = np.array(query_seq)[aln_start: aln_end + 1]
    aln_len = len(np.where(query_aln_seq_array != '-')[0])
    return aln_len

In [97]:
iteration = 1
qresult = qresult_list[iteration - 1]
hit = qresult[1]
print(hit.id)
pir_name = '_'.join(hit.id.split('|')[1: ]) + '_' + str(iteration) + '.pir'
pir_file = data_dir / 'filtering' / pir_name
query_record, template_record = SeqIO.parse(pir_file, 'pir')
query_seq, template_seq = query_record.seq, template_record.seq
template_not_gap_indices = np.where(np.array(template_seq) != '-')[0]
aln_start, aln_end = template_not_gap_indices[0], template_not_gap_indices[-1]
query_aln_seq_array = np.array(query_seq)[aln_start: aln_end + 1]
aln_len = len(np.where(query_aln_seq_array != '-')[0])
print(aln_len)

pdb|4CSU|9
334


In [92]:
np.where(np.array(template_seq) != '-')

(array([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
         49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
         62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
         75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
         88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
        127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
        166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
        179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
        192, 193, 194, 195, 196, 197, 198, 199, 200

In [93]:
np.array(target_seq)[23: 356 + 1]

array(['V', 'D', 'E', 'A', 'S', 'I', 'L', 'V', 'V', 'A', 'G', 'D', 'G',
       'G', 'N', 'G', 'C', 'V', 'S', 'F', 'R', 'R', 'E', 'K', 'Y', 'I',
       'P', 'K', 'G', 'G', 'P', 'D', 'G', 'G', 'D', 'G', 'G', 'D', 'G',
       'G', 'D', 'V', 'W', 'M', 'E', 'A', 'D', 'E', 'N', 'L', 'N', 'T',
       'L', 'I', 'D', 'Y', 'R', 'F', 'E', 'K', 'S', 'F', 'R', 'A', 'E',
       'R', 'G', 'Q', 'N', 'G', 'A', 'S', 'R', 'D', 'C', 'T', 'G', 'K',
       'R', 'G', 'K', 'D', 'V', 'T', 'I', 'K', 'V', 'P', 'V', 'G', 'T',
       'R', 'V', 'I', 'D', 'Q', 'G', 'T', 'G', 'E', 'T', 'M', 'G', 'D',
       'M', 'T', 'K', 'H', 'G', 'Q', 'R', 'L', 'L', 'V', 'A', 'K', 'G',
       'G', 'W', 'H', 'G', 'L', 'G', 'N', 'T', 'R', 'F', 'K', 'S', 'S',
       'V', 'N', 'R', 'T', 'P', 'R', 'Q', 'K', 'T', 'N', 'G', 'T', 'P',
       'G', 'D', 'K', 'R', 'E', 'L', 'L', 'L', 'E', 'L', 'M', 'L', 'L',
       'A', 'D', 'V', 'G', 'M', 'L', 'G', 'M', 'P', 'N', 'A', 'G', 'K',
       'S', 'T', 'F', 'I', 'R', 'A', 'V', 'S', 'A', 'A', 'K', 'P

In [85]:
template_seq

Seq('-----------------------VDEASILVVAGDGGNGCVSFRREKYIPKGGP...---')

In [86]:
list(template_seq)

['-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'V',
 'D',
 'E',
 'A',
 'S',
 'I',
 'L',
 'V',
 'V',
 'A',
 'G',
 'D',
 'G',
 'G',
 'N',
 'G',
 'C',
 'V',
 'S',
 'F',
 'R',
 'R',
 'E',
 'K',
 'Y',
 'I',
 'P',
 'K',
 'G',
 'G',
 'P',
 'D',
 'G',
 'G',
 'D',
 'G',
 'G',
 'D',
 'G',
 'G',
 'D',
 'V',
 'W',
 'M',
 'E',
 'A',
 'D',
 'E',
 'N',
 'L',
 'N',
 'T',
 'L',
 'I',
 'D',
 'Y',
 'R',
 'F',
 'E',
 'K',
 'S',
 'F',
 'R',
 'A',
 'E',
 'R',
 'G',
 'Q',
 'N',
 'G',
 'A',
 'S',
 'R',
 'D',
 'C',
 'T',
 'G',
 'K',
 'R',
 'G',
 'K',
 'D',
 'V',
 'T',
 'I',
 'K',
 'V',
 'P',
 'V',
 'G',
 'T',
 'R',
 'V',
 'I',
 'D',
 'Q',
 'G',
 'T',
 'G',
 'E',
 'T',
 'M',
 'G',
 'D',
 'M',
 'T',
 'K',
 'H',
 'G',
 'Q',
 'R',
 'L',
 'L',
 'V',
 'A',
 'K',
 'G',
 'G',
 'W',
 'H',
 'G',
 'L',
 'G',
 'N',
 'T',
 'R',
 'F',
 'K',
 'S',
 'S',
 'V',
 'N',
 'R',
 'T',
 'P',
 'R',
 'Q',
 'K',
 'T',
 'N',
 'G',
 'T',
 'P',
 'G'